In [402]:
import folium
import pickle
import simplejson
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np

# Make the dataframe

In [356]:
month = 2 #Oct
day_of_week = 1 #Wed
hour = 16 #From 3pm to 4pm

f=open('network_mesh_space=0.0025_month='+str(month)+'_day='+str(day_of_week)+'.pickle','rb')
network = pickle.load(f)
f.close()

In [480]:
n0 = network.loc(axis=0)[:,16,:,23759]

In [481]:
n0.reset_index(level=[0,1,2,3], inplace=True)

In [482]:
gr = n0[['pickup_center',0]].groupby('pickup_center').sum()

In [485]:
gr = gr.sort(0,ascending=False)[0:10]

In [486]:
gr

,0
pickup_center,
23988,25
24879,24
23987,23
24430,21
24653,21
24654,19
24212,18
24877,17
24211,15


In [492]:
for index, row in gr.iterrows():
    print index
    print int(row)

23988
25
24879
24
23987
23
24430
21
24653
21
24654
19
24212
18
24877
17
24211
15
24436
15


In [340]:
trips = {} #{center_id: [outgoing trips, incoming trips]}

count_pick_ups = network.loc(axis=0)[hour,:,:,:]
count_pick_ups = network.loc[(hour,slice(None),slice(None),slice(None)),:]
count_pick_ups_index = count_pick_ups.index.values
for i in count_pick_ups_index:
    try:
        trips[str(int(i[2]))][0] += int(count_pick_ups.ix[i])
    except KeyError:
        trips[str(int(i[2]))] = [int(count_pick_ups.ix[i]),0]

count_drop_offs = network.loc(axis=0)[:,hour,:,:]
count_drop_offs_index = count_drop_offs.index.values
for i in count_drop_offs_index:
    try:
        trips[str(int(i[3]))][1] += int(count_drop_offs.ix[i])
    except KeyError:
        trips[str(int(i[3]))] = [0,int(count_drop_offs.ix[i])]


In [341]:
tk = trips.keys()
tv = trips.values()

map_data = pd.DataFrame({'centers': tk, 'activity' : [i[0]+i[1] for i in tv], 
                         'attractiveness': [i[1]-i[0] for i in tv]})

In [342]:
map_data.activity.max()

160

# Make the grid

In [343]:
f=open('centers_long_lat_id_mesh_space=0.0025.pickle','rb')
grid = pickle.load(f)
f.close()

In [344]:
grid_json = {"type":"FeatureCollection", "features":[]}

In [345]:
add_mesh = 0.0025/2
for index, row in grid.iterrows():
    if str(int(row[2])) in trips.keys():
        popup_content= "Incoming: "+str(trips[str(int(row[2]))][0])+"<br /> outgoing: "+str(trips[str(int(row[2]))][1]) +"<br /> Activity: "+str(trips[str(int(row[2]))][1] + trips[str(int(row[2]))][0]) +"<br /> Attractiveness: "+str(trips[str(int(row[2]))][1] - trips[str(int(row[2]))][0])
        coord = [[row[0]+add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]-add_mesh],[row[0]-add_mesh,row[1]+add_mesh],[row[0]+add_mesh,row[1]+add_mesh]]
        dd = {"type":"Feature","id":str(int(row[2])),
              "properties":{"name":str(int(row[2])),"popupContent":popup_content},
             "geometry":{"type":"Polygon","coordinates":[coord]}
             }
        grid_json['features'].append(dd)

In [302]:
with open('grid.json', 'w') as outfile:
    simplejson.dump(grid_json, outfile)

In [322]:
grid_json = simplejson.dumps(grid_json)

# Make the map

In [348]:
import folium

map_1 = folium.Map(location=[40.74006, -73.98605], zoom_start=12,
                   tiles='Stamen Terrain')
map_1.lat_lng_popover()

In [349]:
map_1.geo_json(geo_str=grid_json, data=map_data,
               columns=['centers', 'activity'],
               threshold_scale=[0, 250, 500, 750, 1000, 2000],
               key_on='feature.id',fill_color='BuPu', fill_opacity=0.5, line_weight=1,
               line_opacity=0.8,line_color='black',
               legend_name='Activity Rate',reset=True)

In [317]:
map_1.create_map(path='nyc.html')

In [209]:
map_html = str(map_1.HTML.encode("ascii", "ignore"))

In [199]:
soup = bs(map_html, 'html.parser') 

In [184]:
map_head = soup.head

In [213]:
map_div = soup.body.div

In [216]:
str(map_div).replace("100%","600px")

'<div id="map" style="width: 600px; height: 600px"></div>'

In [224]:
map_script= soup.body.script

In [355]:
map_1.line?

In [500]:
map_1.map_size = {'width': 100, 'height': 100}

In [503]:
map_1.simple_marker?

In [ ]:
map_1.